# Processing U.S. Bureau of Labor Statistics Datasets

The `Local Area Unemployment Statistics (LAUS)` and `State and Metro Area Employment, Hours, & Earnings (SAE)` datasets were downloaded using the [SAE Databases One Screen option](https://www.bls.gov/sae/data/) and the [LAUS Databases One Screen option]()

The United States Office of Management and Budget (OMB) delineates metropolitan and micropolitan statistical areas according to published standards that are applied to Census Bureau data. The general concept of a metropolitan or micropolitan statistical area is that of a core area containing a substantial population nucleus, together with adjacent communities having a high degree of economic and social integration with that core. Current area delineations are based on OMB Bulletin No. 18-03 effective April 2018.

In [55]:
import json
import pandas as pd

In [56]:
with open('ei_intermediate_file_paths.json') as output_path_file:
    file_paths = json.load(output_path_file)

LAUS_INPUT_PATH = file_paths.get("raw_laus.csv")

SAE_INPUT_PATH = file_paths.get("raw_sae.csv")

BLS_STAGE1_OUTPUT_PATH = file_paths.get("stage1_bls_output.csv")

## Read in the Data

Note: added skiprows due to the format of the file download from the BLS website. Original file formats included in the 02_economic_impact_model/raw directory

In [57]:
laus_df = pd.read_excel(LAUS_INPUT_PATH,skiprows=10)

sae_df = pd.read_excel(SAE_INPUT_PATH,skiprows=12)

/Users/debbiesubocz/anaconda3/envs/512_proj/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/debbiesubocz/anaconda3/envs/512_proj/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [58]:
print("LAUS Dataset Columns")
print(laus_df.columns)

print("First 10 Rows of LAUS Dataset")
print(laus_df.head(10))

LAUS Dataset Columns
Index(['Year', 'Period', 'labor force', 'employment', 'unemployment',
       'unemployment rate'],
      dtype='object')
First 10 Rows of LAUS Dataset
   Year Period  labor force  employment  unemployment  unemployment rate
0  2007    Jan       418651      400707         17944                4.3
1  2007    Feb       417496      400487         17009                4.1
2  2007    Mar       418341      403220         15121                3.6
3  2007    Apr       416759      402919         13840                3.3
4  2007    May       418696      404791         13905                3.3
5  2007    Jun       424835      408624         16211                3.8
6  2007    Jul       426582      410904         15678                3.7
7  2007    Aug       421945      406594         15351                3.6
8  2007    Sep       420835      405049         15786                3.8
9  2007    Oct       421942      405496         16446                3.9


In [59]:
print("SAE Dataset Columns")
print(sae_df.columns)

print("First 10 Rows of SAE Dataset")
print(sae_df.head(10))

SAE Dataset Columns
Index(['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
      dtype='object')
First 10 Rows of SAE Dataset
   Year   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov  \
0  2007  36.5  37.2  36.6  37.2  36.3  36.7  36.9  37.2  38.0  37.2  37.5   
1  2008  37.4  37.8  38.2  37.7  37.3  38.2  36.7  37.7  37.4  37.0  37.5   
2  2009  37.4  37.7  37.7  37.2  37.4  37.9  37.1  37.4  36.8  36.7  37.2   
3  2010  36.5  36.8  37.2  37.7  38.0  37.9  37.4  37.2  36.8  36.9  37.1   
4  2011  36.7  37.7  37.3  37.2  37.4  37.3  36.8  37.1  37.0  37.2  36.9   
5  2012  36.6  36.9  36.5  36.7  36.4  36.9  36.4  36.6  36.7  36.0  35.9   
6  2013  35.5  36.0  35.8  35.5  35.6  36.3  36.0  36.4  36.7  36.1  36.1   
7  2014  35.7  35.4  36.6  36.0  36.2  36.4  35.8  36.2  36.3  36.1  36.7   
8  2015  35.9  35.6  36.1  36.1  36.5  36.7  36.6  37.2  36.4  36.0  36.6   
9  2016  36.3  36.5  36.8  36.8  37.2  36.8  36.6  3

## Process SAE Data

In [60]:
# Step 1: Convert SAE dataset to long format
sae_long = sae_df.melt(id_vars=['Year'], var_name='Month', value_name='curr_sae_hrs')
sae_long['Month'] = sae_long['Month'].str.strip()  # Remove any whitespace in column names

In [61]:
print(sae_long.head(10))

   Year Month  curr_sae_hrs
0  2007   Jan          36.5
1  2008   Jan          37.4
2  2009   Jan          37.4
3  2010   Jan          36.5
4  2011   Jan          36.7
5  2012   Jan          36.6
6  2013   Jan          35.5
7  2014   Jan          35.7
8  2015   Jan          35.9
9  2016   Jan          36.3


In [62]:
# Create a date column
sae_long['date'] = pd.to_datetime(sae_long['Year'].astype(str) + sae_long['Month'], format='%Y%b')


In [63]:
# Sort the DataFrame by the date column in descending order
sae_long = sae_long.sort_values(by='date', ascending=False)


In [64]:
print(sae_long.head(10))

     Year Month  curr_sae_hrs       date
215  2024   Dec           NaN 2024-12-01
197  2024   Nov           NaN 2024-11-01
179  2024   Oct           NaN 2024-10-01
161  2024   Sep          36.0 2024-09-01
143  2024   Aug          36.2 2024-08-01
125  2024   Jul          35.8 2024-07-01
107  2024   Jun          36.4 2024-06-01
89   2024   May          36.0 2024-05-01
71   2024   Apr          35.7 2024-04-01
53   2024   Mar          36.0 2024-03-01


In [65]:
# Step 1: Create a key for the current month-year and the previous month-year
sae_long['key'] = sae_long['Year'].astype(str) + '-' + sae_long['Month']
sae_long['prev_key'] = (sae_long['Year'] - 1).astype(str) + '-' + sae_long['Month']

# Step 2: Rename columns in a copy of the dataframe for merging
sae_lookup = sae_long[['key', 'curr_sae_hrs']].rename(columns={
    'key': 'prev_key',
    'curr_sae_hrs': 'prev_yr_sae_hrs'
})

# Step 3: Merge the current dataframe with the lookup dataframe on 'prev_key'
sae_long = pd.merge(sae_long, sae_lookup, on='prev_key', how='left')

# Drop the temporary keys to clean up the dataframe
sae_long.drop(columns=['key', 'prev_key'], inplace=True)

# Result now includes the explicitly searched previous year's SAE hours
print(sae_long.head(18))


    Year Month  curr_sae_hrs       date  prev_yr_sae_hrs
0   2024   Dec           NaN 2024-12-01             36.0
1   2024   Nov           NaN 2024-11-01             35.5
2   2024   Oct           NaN 2024-10-01             35.9
3   2024   Sep          36.0 2024-09-01             35.9
4   2024   Aug          36.2 2024-08-01             36.5
5   2024   Jul          35.8 2024-07-01             36.6
6   2024   Jun          36.4 2024-06-01             36.6
7   2024   May          36.0 2024-05-01             36.3
8   2024   Apr          35.7 2024-04-01             36.7
9   2024   Mar          36.0 2024-03-01             36.9
10  2024   Feb          36.1 2024-02-01             37.1
11  2024   Jan          35.1 2024-01-01             36.8
12  2023   Dec          36.0 2023-12-01             37.3
13  2023   Nov          35.5 2023-11-01             36.5
14  2023   Oct          35.9 2023-10-01             37.0
15  2023   Sep          35.9 2023-09-01             37.0
16  2023   Aug          36.5 20

## Process the LAUS Data

In [66]:
# Step 1: Create a key for the current month-year and the previous month-year
laus_df['key'] = laus_df['Year'].astype(str) + '-' + laus_df['Period']
laus_df['prev_key'] = (laus_df['Year'] - 1).astype(str) + '-' + laus_df['Period']

# Step 2: Rename columns in a copy of the dataframe for merging
laus_lookup = laus_df[['key', 'labor force', 'unemployment rate']].rename(columns={
    'key': 'prev_key',
    'labor force': 'prev_yr_laus_labor_force',
    'unemployment rate': 'prev_yr_laus_unemployment_rate'
})

# Step 3: Merge the current dataframe with the lookup dataframe on 'prev_key'
laus_df = pd.merge(laus_df, laus_lookup, on='prev_key', how='left')

# Rename columns to match the desired output format
laus_df.rename(columns={
    'labor force': 'curr_laus_labor_force',
    'unemployment rate': 'curr_laus_unemployment_rate'
}, inplace=True)

# Step 4: Create a date column
laus_df['date'] = pd.to_datetime(laus_df['Year'].astype(str) + '-' + laus_df['Period'], format='%Y-%b')

# Step 5: Drop unnecessary columns
laus_df.drop(columns=['employment', 'unemployment', 'key', 'prev_key'], inplace=True)

# Step 6: Sort by date column in descending order
laus_df = laus_df.sort_values(by='date', ascending=False)

# Display the updated dataframe
print(laus_df.head(18))


     Year Period  curr_laus_labor_force  curr_laus_unemployment_rate  \
212  2024    Sep                 449955                          3.3   
211  2024    Aug                 449065                          3.2   
210  2024    Jul                 453547                          3.2   
209  2024    Jun                 451186                          3.4   
208  2024    May                 447798                          2.6   
207  2024    Apr                 449743                          2.5   
206  2024    Mar                 450534                          2.7   
205  2024    Feb                 446352                          2.6   
204  2024    Jan                 442027                          3.3   
203  2023    Dec                 440939                          2.8   
202  2023    Nov                 443648                          2.9   
201  2023    Oct                 444013                          3.1   
200  2023    Sep                 442970                         

## Join the SAE and LAUS Data

In [67]:
# Step 1: Merge the two DataFrames on the 'date' column
merged_df = pd.merge(
    sae_long,
    laus_df,
    on='date',
    how='outer'  # 'outer' for all dates
)

# Step 2: Select and reorder the columns as desired
merged_df = merged_df[[
    'date', 
    'curr_sae_hrs', 
    'curr_laus_labor_force', 
    'curr_laus_unemployment_rate', 
    'prev_yr_sae_hrs', 
    'prev_yr_laus_labor_force', 
    'prev_yr_laus_unemployment_rate'
]]

merged_df = merged_df.sort_values(by='date', ascending=False)

# Display the resulting merged DataFrame
print(merged_df.head(18))


          date  curr_sae_hrs  curr_laus_labor_force  \
215 2024-12-01           NaN                    NaN   
214 2024-11-01           NaN                    NaN   
213 2024-10-01           NaN                    NaN   
212 2024-09-01          36.0               449955.0   
211 2024-08-01          36.2               449065.0   
210 2024-07-01          35.8               453547.0   
209 2024-06-01          36.4               451186.0   
208 2024-05-01          36.0               447798.0   
207 2024-04-01          35.7               449743.0   
206 2024-03-01          36.0               450534.0   
205 2024-02-01          36.1               446352.0   
204 2024-01-01          35.1               442027.0   
203 2023-12-01          36.0               440939.0   
202 2023-11-01          35.5               443648.0   
201 2023-10-01          35.9               444013.0   
200 2023-09-01          35.9               442970.0   
199 2023-08-01          36.5               440564.0   
198 2023-0

In [68]:
# Step 3: Calculate percentage difference for each column
merged_df['pct_diff_sae_hrs'] = ((merged_df['curr_sae_hrs'] - merged_df['prev_yr_sae_hrs']) / merged_df['prev_yr_sae_hrs']) * 100
merged_df['pct_diff_laus_labor_force'] = ((merged_df['curr_laus_labor_force'] - merged_df['prev_yr_laus_labor_force']) / merged_df['prev_yr_laus_labor_force']) * 100
merged_df['pct_diff_laus_unemployment_rate'] = ((merged_df['curr_laus_unemployment_rate'] - merged_df['prev_yr_laus_unemployment_rate']) / merged_df['prev_yr_laus_unemployment_rate']) * 100

# Display the DataFrame with new percentage difference columns
print(merged_df.head(18))

          date  curr_sae_hrs  curr_laus_labor_force  \
215 2024-12-01           NaN                    NaN   
214 2024-11-01           NaN                    NaN   
213 2024-10-01           NaN                    NaN   
212 2024-09-01          36.0               449955.0   
211 2024-08-01          36.2               449065.0   
210 2024-07-01          35.8               453547.0   
209 2024-06-01          36.4               451186.0   
208 2024-05-01          36.0               447798.0   
207 2024-04-01          35.7               449743.0   
206 2024-03-01          36.0               450534.0   
205 2024-02-01          36.1               446352.0   
204 2024-01-01          35.1               442027.0   
203 2023-12-01          36.0               440939.0   
202 2023-11-01          35.5               443648.0   
201 2023-10-01          35.9               444013.0   
200 2023-09-01          35.9               442970.0   
199 2023-08-01          36.5               440564.0   
198 2023-0

In [69]:
# Write the DataFrame to a CSV file
merged_df.to_csv(BLS_STAGE1_OUTPUT_PATH, index=False)

print(f"Merged DataFrame has been saved to {BLS_STAGE1_OUTPUT_PATH}")

Merged DataFrame has been saved to intermediate/stage1-output/stage1_bls_output.csv
